In [1]:
# libraries that we're loading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras import layers, models, applications, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, save_img

from keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

# Define paths
input_dir = '/content/drive/MyDrive/HackOrbit/Original_Images'
output_dir = '/content/drive/MyDrive/HackOrbit/Original_Images_augmented'

# Create output directories if not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Number of augmented images per original
num_augmented_images = 30

# Loop through each category folder
for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)
    if not os.path.isdir(category_path):
        continue

    save_category_path = os.path.join(output_dir, category)
    if not os.path.exists(save_category_path):
        os.makedirs(save_category_path)

    # Loop through each image
    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        img = load_img(image_path)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generate augmented images
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=save_category_path,
                                  save_prefix='aug',
                                  save_format='jpeg'):
            i += 1
            if i >= num_augmented_images:
                break

print("Image augmentation completed successfully!")


Image augmentation completed successfully!


In [3]:

# Loading the data and processing it so that we can transfer it into the model
root_dir = "/content/drive/MyDrive/HackOrbit/Original_Images_augmented"
classification_names = ["DirtyFloor", "OverflowingDustbins", "TrashPresence", "WaterLeaks"]

# Function to convert the images into vectorize form and provide them label
def load_images_and_labels(root_dir, classification_names, img_size=(224,224)):
    # Creating an empty list to store the images and their corresponding labels
    data = []

    # Loading the images from each folder present into the brain folder
    for i, classification_name in enumerate(classification_names):
        # full path to the disease folder
        classification_path = os.path.join(root_dir, classification_name)

        # loading image files in the current disease folder
        image_files = os.listdir(classification_path)

        # getting each image file in the current folder
        for image_file in image_files:

            # Checking image
            if image_file.lower().endswith((".jpg", ".png", ".jpeg")):
                # full path to the image file
                image_path = os.path.join(classification_path, image_file)

                # Loading the image using OpenCV
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Warning: Could not read image: {image_path}")
                    continue    # Skip to the next image if loading fails

                # resizing the image to a consistent size and converting into the grayscale image
                img = cv2.resize(img, img_size)
                label = i
                data.append((img, label))

    print(f"Total images loaded: {len(data)}")
    return data

In [4]:
# function to apply remove_noise and split the initial corpus into vectors of image and labels
def preprocess_data(data):
    corpus = []
    labels = []
    for img, label in data:
        # appending the denoised image to the new corpus
        corpus.append(img)
        labels.append(label)
    return np.array(corpus), np.array(labels)

In [5]:
# Function to prepare the dataset for training the model
def prepare_dataset(images, labels, batch_size=32, shuffle=True):
    # normalizing the images and expand dims for channel
    images = images.astype('float32') / 255.0
    images = np.expand_dims(images, axis=-1)
    labels = to_categorical(labels, num_classes=len(classification_names))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


In [6]:
class ClassifierOptimized:
    # constructor to initalise the values
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    # function to buil the model
    def build_cnn(self):
        inputs = layers.Input(shape=self.input_shape)
        x = layers.Conv2D(32, (7, 7), padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)

        # adjust the residual connection to match the number of filters
        residual = layers.Conv2D(64, (1, 1), padding='same')(x)  # Matching the filter size
        residual = layers.BatchNormalization()(residual)

        x = layers.Conv2D(64, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.add([x, residual])  # Adding the residual connection

        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Dense(64 // 16, activation='relu')(se)
        se = layers.Dense(64, activation='sigmoid')(se)
        se = layers.Reshape((1, 1, 64))(se)
        x = layers.multiply([x, se])

        x = layers.Conv2D(128, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.5)(x)

        outputs = layers.Dense(self.num_classes, activation='softmax')(x)

        # compliling the model together with the inputs and outputs layers
        model = models.Model(inputs, outputs)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')],
        )
        return model

In [7]:
# loading and preprocess data
data = load_images_and_labels(root_dir, classification_names)
images, labels = preprocess_data(data)

Total images loaded: 1270


In [31]:
# dividing the data into train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42, stratify=labels)

In [32]:
X_train.shape, X_test.shape

((1143, 224, 224, 3), (127, 224, 224, 3))

In [33]:
# preapring the dataset
batch_size = 32
train_ds = prepare_dataset(X_train, y_train, batch_size=batch_size, shuffle=True)
test_ds = prepare_dataset(X_test, y_test, batch_size=batch_size, shuffle=False)

In [34]:

# building the model
input_shape = (224, 224, 3)
num_classes = len(classification_names)
classifier = ClassifierOptimized(input_shape, num_classes)
model = classifier.build_cnn()



In [35]:
# training model
history = model.fit(train_ds, epochs=30, validation_data=test_ds)

# evaluateing the model
loss, accuracy, auc = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}, Test AUC: {auc:.4f}")

Epoch 1/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 18s 281ms/step - accuracy: 0.3858 - auc: 0.6268 - loss: 1.3250 - val_accuracy: 0.2598 - val_auc: 0.5545 - val_loss: 1.3786
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - accuracy: 0.6212 - auc: 0.8685 - loss: 0.9673 - val_accuracy: 0.2598 - val_auc: 0.5744 - val_loss: 1.4100
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.7162 - auc: 0.9173 - loss: 0.7982 - val_accuracy: 0.2598 - val_auc: 0.6116 - val_loss: 1.5810
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - accuracy: 0.7506 - auc: 0.9342 - loss: 0.7005 - val_accuracy: 0.2598 - val_auc: 0.6289 - val_loss: 1.9306
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.8082 - auc: 0.9591 - loss: 0.5859 - val_accuracy: 0.2598 - val_auc: 0.6103 - val_loss: 2.4044
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.8217 - auc: 0.9672 - loss: 0.5353 - val_accuracy: 0.2598 - val_auc: 0.6003 - val_loss: 2.8690
Epoch 7/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
# saving the model so taht we can use it to use in backend
model.save('/content/drive/MyDrive/HackOrbit/hackrbitmodel.keras') 